In [84]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [85]:
data = pd.read_csv("Job/job.csv")
print(data) 

             company_name                                    job_description  \
0                  Google  minimum qualifications\nbachelors degree or eq...   
1                   Apple  description\nas an asc you will be highly infl...   
2                 Netflix  its an amazing time to be joining netflix as w...   
3             Robert Half  description\n\nweb designers looking to expand...   
4               TrackFive  at trackfive weve got big goals were on a miss...   
..                    ...                                                ...   
848               Menards  job description\n\nparttime\n\nmake big money ...   
849                Parker  responsibilities\nparkers internship program w...   
850        Borgen Project   the borgen project is an innovative national ...   
851  Wyndham Destinations  put the world on vacation\n\nat wyndham destin...   
852               Aerotek  this job handles customer inquiries by telepho...   

                                       

In [86]:
import pandas as pd
import re
import string
import nltk

# Télécharger les stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

# Fonction de nettoyage du texte
def clean_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer les caractères numériques
    text = ''.join([word for word in text if not word.isdigit()])
    # Supprimer les URLs
    text = re.sub('http\S+\s*', ' ', text)
    # Supprimer les hashtags
    text = re.sub('#\S+', '', text)
    # Supprimer les mentions
    text = re.sub('@\S+', ' ', text)
    # Supprimer les caractères de ponctuation
    text = "".join([word for word in text if word not in string.punctuation])
    # Supprimer les caractères non ASCII
    text = re.sub(r'[^\x00-\x7f]', r' ', text)
    # Supprimer les stopwords
    text = " ".join([word for word in text.split() if word not in stopwords])
    # Supprimer les espaces supplémentaires
    text = re.sub('\s+', ' ', text).strip()
    return text

# Charger le fichier CSV
data = pd.read_csv("Job/job.csv")

# Sélectionner uniquement les colonnes nécessaires
data = data[['job_description', 'position_title']]

# Appliquer le nettoyage sur les colonnes
data['job_description'] = data['job_description'].astype(str).apply(clean_text)
data['position_title'] = data['position_title'].astype(str).apply(clean_text)

# Sauvegarder les données nettoyées dans un nouveau fichier CSV
data.to_csv("Job/jobClean.csv", index=False)


<>:17: SyntaxWarning: invalid escape sequence '\S'
<>:19: SyntaxWarning: invalid escape sequence '\S'
<>:21: SyntaxWarning: invalid escape sequence '\S'
<>:29: SyntaxWarning: invalid escape sequence '\s'
<>:17: SyntaxWarning: invalid escape sequence '\S'
<>:19: SyntaxWarning: invalid escape sequence '\S'
<>:21: SyntaxWarning: invalid escape sequence '\S'
<>:29: SyntaxWarning: invalid escape sequence '\s'
C:\Users\pc\AppData\Local\Temp\ipykernel_5748\1503013700.py:17: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('http\S+\s*', ' ', text)
C:\Users\pc\AppData\Local\Temp\ipykernel_5748\1503013700.py:19: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('#\S+', '', text)
C:\Users\pc\AppData\Local\Temp\ipykernel_5748\1503013700.py:21: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('@\S+', ' ', text)
C:\Users\pc\AppData\Local\Temp\ipykernel_5748\1503013700.py:29: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()
[nltk_da

In [87]:
data.head(5)

,job_description,position_title
0,minimum qualifications bachelors degree equiva...,sales specialist
1,description asc highly influential growing min...,apple solutions consultant
2,amazing time joining netflix continue transfor...,licensing coordinator consumer products
3,description web designers looking expand profe...,web designer
4,trackfive weve got big goals mission revolutio...,web developer


In [88]:
data['position_title'].unique()

array(['sales specialist', 'apple solutions consultant',
       'licensing coordinator consumer products', 'web designer',
       'web developer', 'frontend web developer',
       'remote website designer', 'sr web designer', 'senior ui designer',
       'wordpress web developer', 'ui web designer',
       'senior web designer remote', 'chief executive officer',
       'executive vice president chief executive officer ceo medical',
       'ceo', 'ceo positivly', 'chief executive officer ceo consultant',
       'chief executive officer healthcare columbus', 'ceo coach',
       'assistant vice president premier relationship manager',
       'digital marketing specialist', 'senior marketing specialist',
       'performance marketing specialist paid media',
       'software engineer reno nv', 'software engineer',
       'entry level software engineer', 'associate software engineering',
       'software engineer javascript backend', 'senior software engineer',
       'warehouse manager', 'w

In [89]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_top_matches(specific_domain, k=5):

    # Charger les fichiers CSV
    job_desc_df =  pd.read_csv("Job/jobClean.csv")  # Fichier contenant les descriptions de poste
    resume_df = pd.read_csv("Resume_Clean/Concate_Resume.csv")  # Fichier contenant les CV

    # Filtrer les descriptions pour le domaine spécifique
    filtered_job_desc_df = job_desc_df[job_desc_df['position_title'] == specific_domain]

    if filtered_job_desc_df.empty:
        print(f"Aucune description trouvée pour le domaine : {specific_domain}")
        return None

    # Préparer les descriptions et CV pour ce domaine
    job_descriptions = filtered_job_desc_df['job_description'].fillna("")
    resumes = resume_df['Resume'].fillna("")

    # Vectorisation TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_job_desc_vectors = vectorizer.fit_transform(job_descriptions)
    tfidf_resume_vectors = vectorizer.transform(resumes)

    # Calculer la similarité cosinus
    similarity_matrix = cosine_similarity(tfidf_job_desc_vectors, tfidf_resume_vectors)

    # Initialiser une liste pour stocker les résultats
    results = []

    for i, similarities in enumerate(similarity_matrix):
        position_title = filtered_job_desc_df.iloc[i]['position_title']  # Titre du poste
        top_k_indices = np.argsort(similarities)[::-1][:k]  # Indices des k meilleures similarités

        for j in top_k_indices:
            resume_id = resume_df.iloc[j]['ID']  # ID du CV
            resume_category = resume_df.iloc[j]['Category']
            similarity_score = similarities[j] * 100  # Convertir en pourcentage

            # Ajouter le résultat dans la liste
            results.append({
              'Resume ID': resume_id,
              'Resume Domain': resume_category,
              'Similarity (%)': f"{similarity_score:.2f}%",  # Formater en pourcentage
              'Job Domain': position_title,
            })

    # Convertir les résultats en DataFrame
    result_df = pd.DataFrame(results)

    # Trier les résultats par similarité décroissante
    result_df = result_df.sort_values(by='Similarity (%)', ascending=False)

    # Afficher les k meilleures correspondances pour le domaine spécifié
    top_k_results = result_df.head(k)
    print(top_k_results)

    # Sauvegarder les résultats pour le domaine spécifique
    #top_k_results.to_csv(f"top_{k}_similarity_results_{specific_domain.replace(' ', '_')}.csv", index=False)

    return top_k_results


In [90]:
# Domaine spécifique à analyser
specific_domain = "remote website designer"
# Appel de la fonction
top_results = get_top_matches(specific_domain) 

    Resume ID Resume Domain Similarity (%)               Job Domain
0        72.0      Designer         73.40%  remote website designer
1  91153752.0      Designer         70.77%  remote website designer
2        48.0      Designer         70.05%  remote website designer
3  24583187.0      Designer         69.92%  remote website designer
4        58.0      Designer         69.73%  remote website designer


In [79]:
# Domaine spécifique à analyser
specific_domain = "software developer"
# Appel de la fonction
top_results = get_top_matches(specific_domain)

    Resume ID           Resume Domain Similarity (%)          Job Domain
10      266.0  Information Technology         43.04%  software developer
11      247.0  Information Technology         40.92%  software developer
12      171.0  Information Technology         40.72%  software developer
13      185.0  Information Technology         40.65%  software developer
14      204.0  Information Technology         40.59%  software developer


In [81]:
# Domaine spécifique à analyser
specific_domain = "restaurant operations manager"
# Appel de la fonction
top_results = get_top_matches(specific_domain)

   Resume ID     Resume Domain Similarity (%)                     Job Domain
0      140.0         managment         55.14%  restaurant operations manager
1      158.0         managment         52.85%  restaurant operations manager
2      772.0  food & beverages         49.95%  restaurant operations manager
3      780.0  food & beverages         49.67%  restaurant operations manager
4      793.0  food & beverages         49.53%  restaurant operations manager


In [80]:
# Domaine spécifique à analyser
specific_domain = "talent acquisition manager"

# Appel de la fonction
top_results = get_top_matches(specific_domain)

   Resume ID     Resume Domain Similarity (%)                  Job Domain
0        5.0                HR         49.20%  talent acquisition manager
1       12.0                HR         47.01%  talent acquisition manager
2      533.0  health & fitness         46.15%  talent acquisition manager
3       29.0                HR         45.05%  talent acquisition manager
4      911.0       Engineering         45.05%  talent acquisition manager


In [82]:
# Domaine spécifique à analyser
specific_domain = "animation programmer"

# Appel de la fonction
top_results = get_top_matches(specific_domain)

   Resume ID           Resume Domain Similarity (%)            Job Domain
0     1196.0                Designer         51.94%  animation programmer
1      171.0  Information Technology         48.39%  animation programmer
2      916.0             Engineering         47.65%  animation programmer
3      993.0             Engineering         47.62%  animation programmer
4      915.0             Engineering         47.62%  animation programmer


In [83]:
# Domaine spécifique à analyser
specific_domain = "programmer analyst"
# Appel de la fonction
top_results = get_top_matches(specific_domain)

Aucune description trouvée pour le domaine : programmer analyst
